# Crime data in context to immigration and population data

Two things that impact the data is the:

* 2015 - 2020 there was a change in policing strategy to Frontline 2020
* in 2020 there was a sharp dip due to COIVD-19
* There does look like there is a seasonal swing with the data but could not find the cause doesn't seem to be weather related.

https://www.police.wa.gov.au/Crime/CrimeStatistics
https://www.abs.gov.au/statistics/people/population/national-state-and-territory-population/latest-release
https://www.abs.gov.au/statistics/people/population/migration-australia/latest-release

https://www.perthnow.com.au/news/wa/public-satisfaction-with-police-up-but-new-model-needs-improving-ng-36a3bbb3119fc33453283c45345f159a
https://www.abc.net.au/news/2016-02-11/wa-police-commissioner-backs-down-on-new-policing-model/7159736
https://wamnnews.com.au/news/wa-police-overhauls-front-line-model-to-combat-rising-crime-rate/
https://thewest.com.au/politics/law-and-order/secret-police-files-liza-harvey-says-frontline-2020-policing-model-was-never-given-a-chance-ng-b881236411z
https://www.abc.net.au/news/2020-04-08/coronavirus-shutdown-sees-crime-rate-drop-in-wa/12132410

In [1]:
# Dependencies
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.stats import linregress

In [ ]:
## functions ##

# load and clean crime datasets
def load_crime_data(datasets, path):
    df = pd.DataFrame()
    for dataset in datasets:
        # load the csv file
        file_path = Path(f"{path}{dataset}.csv")
        temp_df = pd.read_csv(file_path)
        # add the region
        temp_df["region"] = dataset
        df = pd.concat([df, temp_df])
    return df

In [ ]:
# Load crime data set into pandas

regions = ["goldfields-esperance", "mandurah", "mirrabooka", "south_west", "armadale", "great_southern", "perth", "wheatbelt", "cannington", "joondalup", "mid_west-gascoyne", "plibara", "fremantle", "kimberley", "midland"]
metro_region = ["metropolitan", "regional"]

crime_all_df = load_crime_data(regions, 'crime-data/wa_crime_')
crime_metro_region_all_df = load_crime_data(metro_region, 'crime-data/wa_crime_')

crime_metro_region_all_df.head()

In [ ]:
# convert the dates
dates = pd.to_datetime(crime_metro_region_all_df['Month and Year'], format='%b-%y')
last_days = dates + pd.offsets.MonthEnd(1)
crime_metro_region_all_df['Month and Year'] = last_days
crime_metro_region_all_df = crime_metro_region_all_df.rename(columns={'Month and Year': 'Date'})
crime_metro_region_all_df = crime_metro_region_all_df.set_index('Date')

In [ ]:
file_path = Path("other-data/immigration-data.csv")
immigration_df = pd.read_csv(file_path)
immigration_df = immigration_df[['DateTime', 'WA']]
dates = pd.to_datetime(immigration_df['DateTime'], format='ISO8601')
immigration_df['DateTime'] = dates
immigration_df = immigration_df.rename(columns={'DateTime': 'Date', 'WA': 'Total WA immigration'})
immigration_df['Date1'] = immigration_df['Date'].copy()

immigration_df = immigration_df.set_index('Date')

immigration_df.head()

In [ ]:

# change to quarterly data & resize the dataset to fit
crime_quarterly_df = crime_metro_region_all_df.resample('Q').sum()
crime_quarterly_df = crime_quarterly_df.loc['2010-06-01':'2022-07-01']

#remove the unwanted columns # crime_metro_region_all_df.columns.to_list() # to get all the columns
crime_quarterly_df = crime_quarterly_df.drop(columns=[
 #'Homicide Total',
 'Murder',
 'Attempted / Conspiracy to Murder',
 'Manslaughter',
 'Driving Causing Death',
 #'Recent Sexual Offence Total',
 'Sexual Assault',
 'Non-Assaultive Sexual Offences',
 #'Historical Sexual Offence Total',
 'Sexual Assault.1',
 'Non-Assaultive Sexual Offences.1',
 #'Assault (Family) Total',
 'Serious Assault (Family)',
 'Common Assault (Family)',
 #'Assault (Non-Family) Total',
 'Serious Assault (Non-Family)',
 'Common Assault (Non-Family)',
 'Assault Police Officer',
 #'Threatening Behaviour (Family) Total',
 'Threatening Behaviour (Family)',
 'Possess Weapon to Cause Fear (Family)',
 #'Threatening Behaviour (Non-Family) Total',
 'Threatening Behaviour (Non-Family)',
 'Possess Weapon to Cause Fear (Non-Family)',
 #'Deprivation of Liberty Total',
 'Kidnapping / Child Stealing',
 'Deprivation of Liberty',
 #'Robbery Total',
 'Robbery (Business)',
 'Robbery (Non-Business)',
 'Unnamed: 31',
 #'Burglary Total',
 'Burglary (Dwelling)',
 'Burglary (Non-Dwelling)',
 'Stealing of Motor Vehicle',
 #'Stealing Total',
 'Stealing From Motor Vehicle (Contents or Parts)',
 'Stealing From Retail Premises (Shoplift)',
 'Stealing From Dwelling',
 'Stealing From Other Premises or Place',
 'Stealing as a Servant',
 'Stealing (Not Elsewhere Classified)',
 #'Property Damage Total',
 'Criminal Damage',
 'Damage',
 #'Arson Total',
 'Cause Bushfire',
 'Cause Damage by Fire',
 'Other Fire Related Offences',
 'Unnamed: 50',
 'Unnamed: 51',
 #'Drug Offences Total',
 'Drug Dealing',
 'Drug Possession',
 'Possession of Drug Paraphernalia',
 'Cultivate or Manufacture Drugs',
 'Other Drug Offences',
 #'Receiving and Possession of Stolen Property Total',
 'Possess Stolen Property',
 'Receiving Stolen Property',
 'Regulated Weapons Offences',
 'Unnamed: 62',
 'Graffiti',
 #'Fraud & Related Offences Total',
 'Forgery',
 'Fraud (Credit Card)',
 'Fraud (Not Elsewhere Classified)',
 #'Breach of Violence Restraint Order Total',
 'Breach of Family Violence Restraint Order',
 'Breach of Violence Restraint Order',
 'Breach of Police Order',
 'Total Selected Miscellaneous Offences',
 'region'
 ])

# add all the crime together
crime_quarterly_df['Total Crime'] = crime_quarterly_df.sum(axis=1)
crime_quarterly_df.head()

In [ ]:
#Summary Statistics
# max, min, mean, mediam, variance, standard deviation for each year
crime_yearly_descriptive_stats_df = crime_quarterly_df.copy()
crime_yearly_descriptive_stats_df = crime_yearly_descriptive_stats_df.groupby(crime_yearly_descriptive_stats_df.index.year)['Total Crime'].agg(['max', 'min', 'mean', 'median', 'var', 'std'])
crime_yearly_descriptive_stats_df

In [ ]:

# merge the datasets based on dates
immigration_vs_crime_df = crime_quarterly_df.merge(immigration_df, on="Date").copy()
immigration_vs_crime_df = immigration_vs_crime_df[['Total Crime', 'Total WA immigration', 'Date1']]
immigration_vs_crime_df['Total Crime'] = immigration_vs_crime_df['Total Crime'].astype(int)
immigration_vs_crime_df.head()


In [ ]:
# create a line graph
fig, ax1 = plt.subplots() 

fig.set_size_inches(20, 5)

ax1.set_xlabel('Date') 
ax1.set_ylabel('Total Crime', color = 'red') 
ax1.plot(immigration_vs_crime_df['Date1'],immigration_vs_crime_df['Total Crime'], color = 'red') 
ax1.tick_params(axis ='y', labelcolor = 'red') 

# Adding Twin Axes

ax2 = ax1.twinx() 
  
ax2.set_ylabel('WA Immigration', color = 'blue') 
ax2.plot(immigration_vs_crime_df['Date1'],immigration_vs_crime_df['Total WA immigration'], color = 'blue')
ax2.tick_params(axis ='y', labelcolor = 'blue') 


# set monthly locator
ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
# set formatter
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'))
# set font and rotation for date tick labels
plt.gcf().autofmt_xdate()
# Show plot
plt.show()

In [ ]:
# Plot out rooms versus median house price
y_values = immigration_vs_crime_df['Total Crime']
x_values = immigration_vs_crime_df['Total WA immigration']

# Print out the r-squared value along with the plot.
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(30000,73000),fontsize=10,color="red")
plt.ylabel('Total WA Crime')
plt.xlabel('Total WA Immigration')
print(f"The r-squared is: {rvalue**2}")
plt.show()

In [ ]:
file_path = Path("other-data/population-data.csv")
population_df = pd.read_csv(file_path)

population_df = population_df.rename(columns={'Unnamed: 0': 'Date', 'Estimated Resident Population ;  Persons ;  Western Australia ;': 'WA Population'})
population_df = population_df.iloc[10:]

dates = pd.to_datetime(population_df['Date'], format='%b-%Y')
last_days = dates + pd.offsets.MonthEnd(1)
population_df['Date'] = last_days
population_df['Date1'] = population_df['Date'].copy()

population_df = population_df[['Date', 'WA Population', 'Date1']]
population_df = population_df.set_index('Date')

population_df.head()

In [ ]:
population_vs_crime_df = crime_quarterly_df.merge(population_df, on="Date").copy()

population_vs_crime_df = population_vs_crime_df[['Total Crime', 'WA Population', 'Date1']]
population_vs_crime_df['Total Crime'] = population_vs_crime_df['Total Crime'].astype(int)
population_vs_crime_df['WA Population'] = population_vs_crime_df['WA Population'].astype(int)

population_vs_crime_df['WA Population'] = population_vs_crime_df['WA Population']

population_vs_crime_df.head()


In [ ]:
fig, ax1 = plt.subplots() 

fig.set_size_inches(20, 5)

ax1.set_xlabel('Date') 
ax1.set_ylabel('Total Crime', color = 'red')
ax1.plot(population_vs_crime_df['Date1'],population_vs_crime_df['Total Crime'], color = 'red')
ax1.tick_params(axis ='y', labelcolor = 'red')

# Adding Twin Axes

ax2 = ax1.twinx()
  
ax2.set_ylabel('WA Population', color = 'blue') 
ax2.plot(population_vs_crime_df['Date1'],population_vs_crime_df['WA Population'], color = 'blue')
ax2.tick_params(axis ='y', labelcolor = 'blue') 
 
# set monthly locator
ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
# set formatter
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'))
# set font and rotation for date tick labels
plt.gcf().autofmt_xdate()

# Show plot

plt.show()

In [ ]:
# Plot out rooms versus median house price
y_values = population_vs_crime_df['Total Crime']
x_values = population_vs_crime_df['WA Population']

# Print out the r-squared value along with the plot.
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(2300000, 73000),fontsize=10,color="red")
plt.ylabel('Total WA Crime')
plt.xlabel('Total WA Population')
print(f"The r-squared is: {rvalue**2}")
plt.show()